## Imports

In [ ]:
from http.cookies import SimpleCookie

import requests
import httpx
import aiohttp
from curl_cffi import requests as curl_requests
from curl_cffi.requests import AsyncSession


## Constants

In [ ]:
DEFAULT_USER_AGENT = (
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/134.0.0.0 Safari/537.36"
)

DEFAULT_USER_HEADERS = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": "en,zh;q=0.9,zh-CN;q=0.8",
    "User-Agent": DEFAULT_USER_AGENT,
    "Connection": "keep-alive",
}
LENGTH = 500


## Helper Functions

In [ ]:
def cookie_str_to_dict(cookie_str: str) -> dict[str, str]:
    """Convert a cookie string into a dictionary usable by requests/httpx/aiohttp."""
    cookie = SimpleCookie()
    cookie.load(cookie_str)
    return {key: morsel.value for key, morsel in cookie.items()}

def print_response(content: str, status_code: int, length: int = LENGTH) -> None:
    """Pretty-print HTTP response info in a consistent way."""
    print(f"Status code: {status_code}")
    if not content:
        print("(empty response)")
        return

    snippet = content[:length]
    print(f"Content preview ({min(len(content), length)} chars):")
    print(snippet)
    if len(content) > length:
        print("... [truncated]")


## GET Requests


In [ ]:

url = "https://example.com"
headers = {
    **DEFAULT_USER_HEADERS,
    # "Referer": "https://www.google.com/",
}
params = {
    # "keyword": "example",
}
cookies_str = r"""
"""  # Optional cookie string

encoding = None
# encoding = "gbk"
# encoding = "utf-8"
http2: bool = True  # for httpx
impersonate = None  # for curl_cffi, e.g. "chrome124", "chrome", None
# impersonate = "chrome"

cookies = cookie_str_to_dict(cookies_str)


In [ ]:
# requests (sync)
resp = requests.get(url, headers=headers, params=params, cookies=cookies)
if encoding:
    resp.encoding = encoding

print("requests")
print_response(resp.text, resp.status_code)



In [ ]:
# httpx (sync)
with httpx.Client(http2=http2, headers=headers, cookies=cookies) as client:
    r = client.get(url, params=params)
    if encoding:
        r.encoding = encoding

    print("httpx sync")
    print_response(r.text, r.status_code)


In [ ]:
# httpx (async)
async def test_httpx_async():
    async with httpx.AsyncClient(http2=http2, headers=headers, cookies=cookies) as client:
        r = await client.get(url, params=params)
        if encoding:
            r.encoding = encoding

        print("httpx async")
        print_response(r.text, r.status_code)

await test_httpx_async()


In [ ]:
# curl_cffi (sync)
r = curl_requests.get(url, headers=headers, cookies=cookies, impersonate=impersonate)
if encoding:
    r.encoding = encoding

print("curl_cffi sync")
print_response(r.text, r.status_code)


In [ ]:
# curl_cffi (async)
async def test_curl_async():
    async with AsyncSession(impersonate=impersonate, headers=headers, cookies=cookies) as s:
        r = await s.get(url)
        if encoding:
            r.encoding = encoding

        print("curl_cffi async")
        print_response(r.text, r.status_code)

await test_curl_async()


In [ ]:
# aiohttp (async)
async def test_aiohttp():
    async with aiohttp.ClientSession(headers=headers, cookies=cookies) as s:
        async with s.get(url, params=params) as r:
            text = await r.text(encoding=encoding)

            print("aiohttp async")
            print_response(text, r.status)

await test_aiohttp()


## POST Requests

In [ ]:

url = "https://example.com/post"
headers = {
    **DEFAULT_USER_HEADERS,
    # "Referer": "https://www.google.com/",
}
data = {
    # "keyword": "example",
}
params = {
    # "keyword": "example",
}
cookies_str = r"""
"""  # Optional cookie string

length = 500
encoding = None
# encoding = "gbk"
# encoding = "utf-8"
http2: bool = True  # for httpx
impersonate = None  # for curl_cffi, e.g. "chrome124", "chrome", None
# impersonate = "chrome"

cookies = cookie_str_to_dict(cookies_str)


In [ ]:
# requests (sync)
resp = requests.post(url, headers=headers, params=params, data=data, cookies=cookies)
if encoding:
    resp.encoding = encoding

print("requests")
print_response(resp.text, resp.status_code)


In [ ]:
# httpx (sync)
with httpx.Client(http2=http2, headers=headers, cookies=cookies) as client:
    r = client.post(url, params=params, data=data)
    if encoding:
        r.encoding = encoding

    print("httpx sync")
    print_response(r.text, r.status_code)


In [ ]:
# httpx (async)
async def test_httpx_async_post():
    async with httpx.AsyncClient(http2=http2, headers=headers, cookies=cookies) as client:
        r = await client.post(url, params=params, data=data)
        if encoding:
            r.encoding = encoding

        print("httpx async")
        print_response(r.text, r.status_code)

await test_httpx_async_post()


In [ ]:
# curl_cffi (sync)
r = curl_requests.post(url, headers=headers, cookies=cookies, data=data, impersonate=impersonate)
if encoding:
    r.encoding = encoding

print("curl_cffi sync")
print_response(r.text, r.status_code)


In [ ]:
# curl_cffi (async)
async def test_curl_async_post():
    async with AsyncSession(impersonate=impersonate, headers=headers, cookies=cookies) as s:
        r = await s.post(url, data=data, params=params)
        if encoding:
            r.encoding = encoding

        print("curl_cffi async")
        print_response(r.text, r.status_code)

await test_curl_async_post()


In [ ]:
# aiohttp (async)
async def test_aiohttp_post():
    async with aiohttp.ClientSession(headers=headers, cookies=cookies) as s:
        async with s.post(url, data=data, params=params) as r:
            text = await r.text(encoding=encoding)

            print("aiohttp async")
            print_response(text, r.status)

await test_aiohttp_post()
